In [26]:
import pandas as pd
import numpy as np

## Functions for cleaing data from kaggle and years 2020-2023 

In [7]:
def clean_val(v):
    bad_values = {"-1", ".", " ", "", "nan"}

    if pd.isna(v):
        return None
    v = str(v).strip()
    return None if v in bad_values else v


#### Combine all free-text into one column and drop the others

In [8]:
def free_text(df):
    df = df.replace("-1", np.nan)
    df = df.replace(-1, np.nan)

    exclude_free_text = {
    "What country do you live in?",
    "What country do you work in?",
    "If you live in the United States, which state or territory do you live in?",
    "If yes, what condition(s) have you been diagnosed with?", # this would be data leakage
    "If maybe, what condition(s) do you believe you have?", # this also
    "What US state or territory do you work in?",
    "What is your age?",
    "What is your gender?",
    "SurveyID",
    "UserID"
    }
    UNIQUE_THRESHOLD = 50       # columns with > 50 unique values are likely free text
    AVG_LENGTH_THRESHOLD = 20   # average length > 20 characters are likely free text

    # add all free text into one array
    
    free_text_cols = []
    for col in df.columns:
        if col in exclude_free_text:
            continue
        num_unique = df[col].nunique()
        avg_len = df[col].astype(str).apply(len).mean()
        if num_unique > UNIQUE_THRESHOLD:
            free_text_cols.append(col)

    for col in free_text_cols:
        print("-", col)
        
    # add all free text into one column & drop the others
    
    df["all_text"] = df[free_text_cols].apply(
        lambda row: " ".join(
            [clean_val(v) for v in row if clean_val(v) is not None]),
        axis=1)
    df["all_text"].replace("", np.nan, inplace=True)
    df.drop(columns=free_text_cols, inplace=True)

    return df

In [78]:
def free_text_2023(df):
    df = df.replace("-1", np.nan)
    df = df.replace(-1, np.nan)

    free_text_cols = [
        "Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions.",
        "Describe the conversation with coworkers you had about your mental health including their reactions.",
        "Describe the conversation your coworker had with you about their mental health (please do not use names).",
        "Describe the conversation you had with your previous employer about your mental health, including their reactions and actions taken to address your mental health issue/questions.",
        "Describe the conversation you had with your previous coworkers about your mental health including their reactions.",
        "Describe the conversation your coworker had with you about their mental health (please do not use names)..1",
        "Describe the circumstances of the supportive or well handled response.",
        "Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.",
        "If there is anything else you would like to tell us that has not been covered by the survey questions, please use this space to do so."
    ]
    
    df["all_text"] = df[free_text_cols].apply(
        lambda row: " ".join(
            [clean_val(v) for v in row if clean_val(v) is not None]),
        axis=1)
    df["all_text"].replace("", np.nan, inplace=True)
    df.drop(columns=free_text_cols, inplace=True)

    return df

#### Keep only columns where more than 50% of respondents answered & Remove respondents (rows) that have too many missing values


In [79]:
def clean_rows(df):
    # Keep only columns where more than 50% of respondents answered
    non_null_counts = df.notna().sum()

    valid_cols = non_null_counts[non_null_counts > df.shape[0] / 2].index
    df = df[valid_cols]
    print("Remaining columns:", df.shape[1])

    # Remove respondents (rows) that have too many missing values
    row_nan_counts = df.isna().sum(axis=1)
    
    threshold = df.shape[1] * 0.5
    
    print("Threshold is", threshold, "rows")
    df = df[row_nan_counts < threshold].reset_index(drop=True)

    print("Remaining rows:", df.shape[0])

    return df


#### Clean gender

In [80]:
def clean_gender(value):
    if pd.isna(value):
        return np.nan
    
    v = str(value).strip().lower()

    # Male
    male = {
        "male", "cishet male", "male-ish", "ostensibly male",
        "masculine", "male/androgynous", "masculino"
    }
    if v in male:
        return "Male"

    # Female
    female = {
        "female", "female-ish", "female-identified",
        "female assigned at birth", "woman-identified",
        "fm", "femmina", "female-bodied; no feelings about gender",
        "gender non-conforming woman"
    }
    if v in female:
        return "Female"

    # Transgender
    trans = [
        "trans woman", "trans man", "trans female", "transgender woman",
        "transitioned, m2f", "mtf", "male (trans, ftm)", "transgender",
        "transfeminine", "other/transfeminine",
        "trans non-binary/genderfluid"
    ]
    if v in trans:
        return "Transgender"

    # Non-binary
    nb = [
        "non-binary", "non binary", "nonbinary", "nb", "enby",
        "genderfluid", "genderqueer", "agender", "androgynous",
        "demiguy", "bigender", "nonbinary/femme", 
        "genderflux demi-girl", "genderqueer demigirl",
        "genderqueer/non-binary", "non-binary and gender fluid",
        "genderfluid (born female)", "genderqueer woman",
        "nb masculine"
    ]
    if v in nb:
        return "Non-binary"

    # Everything else to Other
    return "Other"

#### Filter out ages

In [81]:
def clean_age(df, col, min_age=15, max_age=80):
    
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
    df.loc[
        (df[col] < min_age) | 
        (df[col] > max_age), 
        col
    ] = np.nan
    
    median_age = df[col].median()
    df[col] = df[col].fillna(median_age)
    
    return df

### Clean data from kaggle and years 2020-2023

In [107]:
df_kaggle = pd.read_csv("../data/raw/kaggle_uncleaned.csv")
df20 = pd.read_csv("../data/raw/osmi_mental_health_2020.csv")
df21 = pd.read_csv("../data/raw/osmi_mental_health_2021.csv")
df22 = pd.read_csv("../data/raw/osmi_mental_health_2022.csv")
df23 = pd.read_csv("../data/raw/osmi_mental_health_2023.csv")


osmi = [df_kaggle, df20, df21, df22, df23]
years = ["kaggle", "2020", "2021", "2022", "2023"]

for i, df in enumerate(osmi):
    if i != 0:
        df = df.drop(columns=["#"])
        df['SurveryID'] = int(years[i])
    
    # APPLY FREE TEXT
    if i!=0:
        if int(years[i]) == 2023:
            df = free_text_2023(df)
    else: 
        df = free_text(df)

    # CLEAN ROWS
    df = clean_rows(df)

    # CLEAN GENDER 
    df["Gender_cleaned"] = df["What is your gender?"].apply(clean_gender)
    df.drop(columns=["What is your gender?"], inplace=True)
    df["Gender_cleaned"].value_counts()

    print("")

    # CLEAN AGE
    df = clean_age(df, "What is your age?")

    # add the cleaned df to list
    osmi[i] = df


# turn list into dict
osmi_dict = {years[i]: osmi[i] for i in range(len(osmi))}

- Any additional notes or comments
- Briefly describe what you think the industry as a whole and/or employers could do to improve mental health support for employees.
- Describe the circumstances of the badly handled or unsupportive response.
- Describe the conversation with coworkers you had about your mental health including their reactions.
- Describe the conversation you had with your employer about your mental health, including their reactions and what actions were taken to address your mental health issue/questions.
- Describe the conversation you had with your previous coworkers about your mental health including their reactions.
- Describe the conversation you had with your previous employer about your mental health, including their reactions and actions taken to address your mental health issue/questions.
- Describe the conversation your coworker had with you about their mental health (please do not use names).
- Describe the conversation your coworker had with you about their

C:\Users\karelallik\AppData\Local\Temp\ipykernel_3288\2474315881.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["all_text"].replace("", np.nan, inplace=True)
C:\Users\karelallik\AppData\Local\Temp\ipykernel_3288\2311850890.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [140]:
for name, df in dfs.items():
    print(name, df.shape)

for year in dfs:
    print(year, "→", "all_text" in dfs[year].columns)


kaggle (2911, 38)
2020 (172, 55)
2021 (125, 55)
2022 (154, 56)
2023 (6, 68)
kaggle → True
2020 → True
2021 → True
2022 → True
2023 → True


<br>
<br>

### Find similarity between column names and harmonize with a canonical name.

In [111]:
import glob
from difflib import SequenceMatcher

In [112]:
# Check fuzzy similarity between column names
def similar(a,b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

In [113]:
similar_pairs = []
threshold = 0.80

all_cols = []
for name, df in dfs.items():
    for col in df.columns:
        all_cols.append((name, col))

for i in range(len(all_cols)):
    name1, c1 = all_cols[i]
    for j in range(i+1, len(all_cols)):
        name2, c2 = all_cols[j]
        
        score = similar(c1, c2)
        if score >= threshold and c1 != c2:
            similar_pairs.append((score, c1, c2, name1, name2))
            
# Sort strongest matches first
similar_pairs_sorted = sorted(similar_pairs, key=lambda x: -x[0])

# Preview top matches
#for s, c1, c2, n1, n2 in similar_pairs_sorted[:40]:
    #print(f"{s:.3f} | {n1}:{c1} <--> {n2}:{c2}")

In [114]:
# Finding the shortest column name among each similar group as canonical name
canonical_map = {}

# All pairs are put into one cluster-building process
clusters = []

for score, c1, c2, n1, n2 in similar_pairs_sorted:
    placed = False
    for cluster in clusters:
        if c1 in cluster or c2 in cluster:
            cluster.add(c1)
            cluster.add(c2)
            placed = True
            break
    if not placed:
        clusters.append(set([c1, c2]))

# Choose canonical names
for cluster in clusters:
    canonical = min(cluster, key=len)  # shortest name
    for col in cluster:
        canonical_map[col] = canonical

print("Number of harmonized columns:", len(canonical_map))
#canonical_map

Number of harmonized columns: 52


In [115]:
# Apply harmonization to all datasets
dfs_harmonized = {}

for name, df in dfs.items():           # dfs includes "kaggle", "2020", "2021", "2022", "2023"
    # rename columns via canonical_map
    new_cols = {col: canonical_map.get(col, col) for col in df.columns}
    tmp = df.rename(columns=new_cols)

    # 🔥 KEY FIX: drop duplicated column names (keep first occurrence)
    tmp = tmp.loc[:, ~tmp.columns.duplicated()]

    dfs_harmonized[name] = tmp

# sanity check
for name, df in dfs_harmonized.items():
    dups = df.columns[df.columns.duplicated()]
    print(name, "num duplicate cols after fix:", len(dups))

kaggle num duplicate cols after fix: 0
2020 num duplicate cols after fix: 0
2021 num duplicate cols after fix: 0
2022 num duplicate cols after fix: 0
2023 num duplicate cols after fix: 0


### Find common columns and ensure there are no duplicate columns.

In [116]:
# Compute common columns across harmonized datasets

common_semantic = None
for name, df in dfs_harmonized.items():
    cols = set(df.columns)
    if common_semantic is None:
        common_semantic = cols
    else:
        common_semantic &= cols

common_semantic = sorted(common_semantic)
print("Common columns across Kaggle + all OSMI years:", len(common_semantic))
print(common_semantic)

Common columns across Kaggle + all OSMI years: 28
['Are you self-employed?', 'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?', 'Do you currently have a mental health disorder?', 'Do you have a family history of mental illness?', 'Do you have previous employers?', 'Do you know the options for mental health care available under your employer-provided health coverage?', 'Does your employer offer resources to learn more about mental health disorders and options for seeking help?', 'Does your employer provide mental health benefits as part of healthcare coverage?', 'Gender_cleaned', 'Have you ever sought treatment for a mental health disorder from a mental health professional?', 'Have you had a mental health disorder in the past?', 'Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?', 'Have your previous employers provided m

In [117]:
for name, df in dfs_harmonized.items():
    dups = df.columns[df.columns.duplicated()]
    print(name, "num duplicate cols:", len(dups))
    if len(dups) > 0:
        print("   examples:", list(dups[:10]))

kaggle num duplicate cols: 0
2020 num duplicate cols: 0
2021 num duplicate cols: 0
2022 num duplicate cols: 0
2023 num duplicate cols: 0


### Merge datasets.

In [118]:
# Merge all datasets using the expanded common columns
merged = []

for name, df in dfs_harmonized.items():
    part = df[common_semantic].copy()
    part["source"] = name
    merged.append(part)
final = pd.concat(merged, ignore_index=True)
print("Final merged dataset shape:", final.shape)


Final merged dataset shape: (3368, 29)


In [119]:
final = final.drop(columns=["source"])
print(final.shape, final.columns)

(3368, 28) Index(['Are you self-employed?',
       'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
       'Do you currently have a mental health disorder?',
       'Do you have a family history of mental illness?',
       'Do you have previous employers?',
       'Do you know the options for mental health care available under your employer-provided health coverage?',
       'Does your employer offer resources to learn more about mental health disorders and options for seeking help?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Gender_cleaned',
       'Have you ever sought treatment for a mental health disorder from a mental health professional?',
       'Have you had a mental health disorder in the past?',
       'Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace

In [120]:
final.to_csv("../data/processed/combined_data_with_free_text.csv", index=False)
print("Saved merged file!")
final.shape

Saved merged file!


(3368, 28)

In [121]:
for col in final:
    print(col)

Are you self-employed?
Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?
Do you currently have a mental health disorder?
Do you have a family history of mental illness?
Do you have previous employers?
Do you know the options for mental health care available under your employer-provided health coverage?
Does your employer offer resources to learn more about mental health disorders and options for seeking help?
Does your employer provide mental health benefits as part of healthcare coverage?
Gender_cleaned
Have you ever sought treatment for a mental health disorder from a mental health professional?
Have you had a mental health disorder in the past?
Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?
Have your previous employers provided mental health benefits?
How many employees does your company or organization have?
How wi

<br>
<br>

## Functions related to goal 2:

In [141]:
df_original = final.copy()
#df_original.shape
# for col in df_original:
#      print(col)
#      print(df_original[col].unique())

#### Binary 

In [123]:
def clean_binary(df, cols):
    
    mapping = {"Yes": 1, "No": 0,
                "1": 1, "0": 0,
                1: 1, 0: 0
    }
    for col in cols:
        #print(df[col].unique())
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors="coerce") 
    return df

#### Uncertainty

In [124]:
def clean_uncertainty(df):
    mapping = {
    "Yes": "Yes",
    "No": "No",
    "Maybe": "Maybe",
    "Maybe/Not sure": "Maybe",
    "Possibly": "Maybe",
    "Don't Know": "Unknown",
    "I am not sure": "Unknown",
    "Unknown": "Unknown",
    "I don't know": "Unknown"
    }
    for col in df:
        df[col] = df[col].replace(mapping)

    return df

#### Age

In [125]:
def clean_age(df, col, min_age=15, max_age=80):
    
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
    df.loc[(df[col] < min_age) | (df[col] > max_age), col] = np.nan
    
    median_age = df[col].median()
    df[col] = df[col].fillna(median_age)
    
    return df

#### Openness

In [126]:
def clean_openness(df, col="How willing would you be to share with friends and family that you have a mental illness?"):
    mapping = {
    "Very open": 10,
    "Somewhat open": 7,
    "Neutral": 5,
    "Somewhat not open": 3,
    "Not open at all": 0,
    "Not applicable to me (I do not have a mental illness)": 0
    }
    df[col] = df[col].replace(mapping)
    df[col] = pd.to_numeric(df[col], errors="coerce")  # muudab numbriks
    
    return df

#### Medical leave

In [127]:
def clean_medical_leave(df, col):
    mapping = {
        "Very easy": 5,
        "Somewhat easy": 4,
        "Neither easy nor difficult": 3,
        "Somewhat difficult": 2,
        "Very difficult": 1,
        "Difficult": 1,   
        "Unknown": 0,
        "I don't know": 0
    }
    
    df[col] = df[col].replace(mapping).fillna(0)
    df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

#### Freuency

In [128]:
def clean_frequency(df, cols):
    mapping = {
        "Never": 1,
        "Rarely": 2,
        "Sometimes": 3,
        "Often": 4,
        "Not applicable to me": 0
    }
    for col in cols:
        df[col] = df[col].replace(mapping)
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

#### Discussion

In [129]:
def clean_discussion(df,col):
    mapping = {
        'Yes, they all did':'Yes',
        'Yes': 'Yes',
        'Some did' :'Partial',
        'No': 'No',
        'None did': 'No',
        'Unknown':'Unknown',
        'No, none did': 'No'
    }
    df[col] = df[col].replace(mapping)
    df[col] = df[col].fillna("Unknown")   # NaN -> "Unknown"
    return df


#### Willingness to discuss

In [130]:
def clean_willing_to_discuss(df, col):
    mapping= {
    'Some of my previous employers':'Some',
    'Unknown': 'Unknown',
    'No, at none of my previous employers': 'No',
    'Yes, at all of my previous employers': 'Yes',
    'Yes, all of my previous supervisors': 'Yes',
    'No, none of my previous supervisors': 'No',  
    'Some of my previous supervisors': 'Some'
    }
    df[col] = df[col].replace(mapping)
    return df
    

#### Supportive response

In [131]:
def clean_supportive_response(df, col):
    mapping = {
        "Yes, I experienced": "Yes",
        "Yes, I observed": "Yes",
        "No": "No",
        "Maybe": "Maybe",
        "Unknown": "Unknown",
        "I've always been self-employed": "NA"
    }
    df[col] = df[col].replace(mapping).fillna("Unknown")
    return df

In [132]:
def clean_previous_awareness(df, col):
    mapping = {
        "Yes, I was aware of all of them": "Yes",
        "I was aware of some": "Partial",
        "No, I only became aware later": "No",
        "Unknown": "Unknown",
        "N/A (not currently aware)": "NA",
        "N/A (was not aware)": "NA",
        "N/A (none offered)": "NA"
    }
    df[col] = df[col].replace(mapping).fillna("Unknown")
    return df

In [133]:
binary_cols = [
    "Are you self-employed?",
    "Do you have previous employers?",
    'Have you ever sought treatment for a mental health disorder from a mental health professional?',
    "Is your employer primarily a tech company/organization?"
]

freq_cols = [
    "If you have a mental health disorder, how often do you feel that it interferes with your work when being treated effectively?",
    "If you have a mental health disorder, how often do you feel that it interferes with your work when not being treated effectively (i.e., when you are experiencing symptoms)?"
]
ordinal_cols = [
    "How willing would you be to share with friends and family that you have a mental illness?",
    "If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?",
    "If you have a mental health disorder, how often do you feel that it interferes with your work when being treated effectively?",
    "If you have a mental health disorder, how often do you feel that it interferes with your work when not being treated effectively (i.e., when you are experiencing symptoms)?"
]

categorical_cols = [
    "Do you currently have a mental health disorder?",
    "Do you have a family history of mental illness?",
    "Have you had a mental health disorder in the past?",
    "Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?",
    "Would you feel comfortable discussing a mental health issue with your coworkers?",
    "Would you have been willing to discuss your mental health with your direct supervisor(s)?"
]

### Applying all of the functions

In [134]:
df_cleaned = clean_binary(df_original, binary_cols)
df_cleaned = clean_age(df_cleaned, "What is your age?")
df_cleaned = clean_uncertainty(df_cleaned)
df_cleaned = clean_openness(df_cleaned, "How willing would you be to share with friends and family that you have a mental illness?")
df_cleaned = clean_medical_leave(df_cleaned, "If a mental health issue prompted you to request a medical leave from work, how easy or difficult would it be to ask for that leave?")
df_cleaned = clean_frequency(df_cleaned, freq_cols)
df_cleaned = clean_discussion(df_cleaned, "Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?")
df_cleaned = clean_discussion(df_cleaned, "Does your employer offer resources to learn more about mental health disorders and options for seeking help?")
df_cleaned = clean_discussion(df_cleaned, "Have your previous employers provided mental health benefits?")
df_cleaned = clean_willing_to_discuss(df_cleaned,  "Would you have been willing to discuss your mental health with your direct supervisor(s)?")
df_cleaned = clean_supportive_response(df_cleaned, "Have you observed or experienced a *supportive or well handled response* to a mental health issue in your current or previous workplace?")
df_cleaned = clean_previous_awareness(df_cleaned, "Were you aware of the options for mental health care provided by your previous employers?")

C:\Users\karelallik\AppData\Local\Temp\ipykernel_3288\2847087626.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mapping)
C:\Users\karelallik\AppData\Local\Temp\ipykernel_3288\1846871736.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace(mapping).fillna(0)


#### Ordinal: 'Unknown' and nan --> 0

In [135]:
 for col in ordinal_cols:
     df_cleaned[col] = df_cleaned[col].replace("Unknown", 0).fillna(0)
     df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors="coerce")
    

#### Categorical: 'Unknown' and nan --> 'Unknown'

In [136]:
 for col in categorical_cols:
     df_cleaned[col] = df_cleaned[col].fillna("Unknown")

(3368, 28)


In [142]:
# for col in df_original:
#      print("")
#      print(col)
#      print(df_original[col].unique())
# print(df_cleaned.shape)

### Into a csv file

In [ ]:
df_cleaned.to_csv("../data/processed/Goal2.csv", index=False)